In [2]:
import torch
import onnx
import onnxruntime
import tensorflow as tf
import tf2onnx
import numpy as np
import os

# Define file paths
TORCHSCRIPT_MODEL_PATH = "mobilenet_distilled_v2.pt"
ONNX_MODEL_PATH = "mobilenet_distilled.onnx"
TF_SAVED_MODEL_DIR = "mobilenet_tf_saved_model"
TFLITE_MODEL_PATH = "mobilenet_distilled.tflite"


In [3]:
#export to onnx
print("Exporting to ONNX...")
dummy_input = torch.randn(1, 3, 256, 256)

torch.onnx.export(
    model,
    dummy_input,
    ONNX_MODEL_PATH,
    input_names=["input"],
    output_names=["output"],
    opset_version=11,
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
)

print(f"ONNX model saved to {ONNX_MODEL_PATH}")


Exporting to ONNX...
ONNX model saved to mobilenet_distilled.onnx


In [4]:
#Check ONNX Model
print("Checking ONNX model...")
onnx_model = onnx.load(ONNX_MODEL_PATH)
onnx.checker.check_model(onnx_model)
print("ONNX model is valid.")


Checking ONNX model...
ONNX model is valid.


In [7]:
from onnx_tf.backend import prepare

onnx_model = onnx.load("mobilenet_distilled.onnx")
tf_rep = prepare(onnx_model)
tf_rep.export_graph("mobilenet_tf_saved_model")


c:\Users\arpitasharma2\AppData\Local\anaconda3\envs\tflite-env\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\arpitasharma2\AppData\Local\anaconda3\envs\tflite-env\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If yo


INFO:tensorflow:Assets written to: mobilenet_tf_saved_model\assets


INFO:tensorflow:Assets written to: mobilenet_tf_saved_model\assets


In [8]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("mobilenet_tf_saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("mobilenet_distilled.tflite", "wb") as f:
    f.write(tflite_model)

In [9]:
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input shape:", input_details[0]['shape'])
print("Output shape:", output_details[0]['shape'])


Input shape: [  1   3 256 256]
Output shape: [1 2]
